In [ ]:
import pandas as pd
import pathlib
import ipytest
ipytest.autoconfig()
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import os
import sys 
sys.path.append("/net/data.isilon/ag-cherrmann/nschmidt/project/parse_xml_for_VAE/xml_data/")
sys.path.append("/net/data.isilon/ag-cherrmann/nschmidt/project/parse_xml_for_VAE/xml_data_t/")
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import re

In [ ]:
path_to_xml = "/net/data.isilon/ag-cherrmann/nschmidt/project/parse_xml_for_VAE/xml_data/"
path_to_xml_t = "/net/data.isilon/ag-cherrmann/nschmidt/project/parse_xml_for_VAE/xml_data_t/"

paths = list(pathlib.Path(path_to_xml).glob("*.csv"))
paths_t = list(pathlib.Path(path_to_xml_t).glob("*.csv"))


In [ ]:
def plot_distribution(paths: list, measurement: str):
    total_fig = make_subplots(rows=len(paths), cols=1)
    total_fig_data = []

    for path in paths: 
        df = pd.read_csv(path, index_col=[0])
        df = df.loc[df['Volume'] == measurement]
        rois = df.columns[1:]   
        
        fig = go.Figure()

        for roi in rois:
            fig.add_trace(go.Box(y=df[roi], name=roi))

        fig.update_layout(boxmode='group') #to group boxes of the same type.
        fig.update_traces(boxpoints='all', jitter=.3)
        
        total_fig_data.append(fig)
    
    for idx, fig in enumerate(total_fig_data):
        for trace in fig.data:
            total_fig.add_trace(trace, row=idx+1, col=1)
    
    total_fig.update_layout(height=1200)
    total_fig.show()

plot_distribution(paths_t, measurement="Vgm")


In [ ]:
path = "/net/data.isilon/ag-cherrmann/nschmidt/project/parse_xml_for_VAE/xml_data_t/Aggregated_suit_t.csv"

df = pd.read_csv(path, index_col=[0])
rois = df.columns[2:]

fig = make_subplots(rows=1, cols=len(rois))
for i, roi in enumerate(rois):
    fig.add_trace(
    go.Box(y=df[roi], name=roi), row=1, col=i+1)

fig.update_traces(boxpoints='all', jitter=.3)
fig.update_layout()

In [ ]:
def analyze_df(df: pd.DataFrame) -> dict: 
    data_dict = {}
    rois = df.columns[1:]
    for roi in rois:
        roi_data = df[roi]
        
        data_dict[roi] = {
            "mean":roi_data.mean(),
            "median":roi_data.median(),
            "quantiles": roi_data.quantile(q=[0.25, 0.5, 0.75]).to_list(),
            "min": roi_data.min(), 
            "max": roi_data.max(),
            "var": roi_data.var()
        }
        
    return data_dict

def analyze_features(paths: list):
    all_data = {}
    for path in paths_t: 
        path_str = str(path)
        df = pd.read_csv(path, index_col=[0]).copy()
        all_data[path_str] = {}

        for measurement in ["Vgm", "Vwm"]:
            df_m = df.loc[df['Volume'] == measurement].copy()
            
            if df_m.shape[0] == 0:
                continue

            data_dict = analyze_df(df_m)
            all_data[path_str][measurement] = data_dict
    return all_data

In [ ]:
all_data = analyze_features(paths=paths_t)

atlas = "suit"
measurement = "Vgm"

path = f"/net/data.isilon/ag-cherrmann/nschmidt/project/parse_xml_for_VAE/xml_data_t/Aggregated_{atlas}_t.csv"

rois = [roi for roi in all_data[path][measurement]]
variances = [all_data[path][measurement][roi]["var"] for roi in rois]

sort_index = np.argsort(variances)
sorted_rois = np.array(rois)[sort_index].tolist()
print(f"Variances: {variances}")
print(f"ROIs: {rois}")
print(f"Sorted ROIs (ascending): {sorted_rois}")

In [ ]:
import pandas as pd
import numpy as np 

def normalize_and_scale_df(df: pd.DataFrame) -> pd.DataFrame:
    # Normalizes the columns (patient volumes) by Min-Max Scaling and scales the rows (ROIs) with Z-transformation.

    df_copy = df.copy()
    column_sums = df_copy.sum()
    
    # Apply the formula: ln((10000*value)/sum_values + 1) "Log transformation"
    # Alternatively for Min-Max Scaling: df_copy/df_copy.max() - Problem: Some rows have std = 0
    transformed_df = np.log((10000 * df_copy) / column_sums + 1)
    
    norm_copy = transformed_df.copy()

    cols = norm_copy.columns.get_level_values(-1).tolist()
    unique_cols = list(set(cols))
    
    if len(unique_cols) > 0:
        for col_type in unique_cols:
            cols_to_scale = [col for col in norm_copy.columns if col[-1] == col_type] 
            print(cols_to_scale)
            print(norm_copy[cols_to_scale])
            print(norm_copy.apply(lambda x: pd.Series((x-x.mean())/x.std()) if x.std() > 0 else pd.Series([0]*len(x)), axis="columns").head)
    # else:
    #     norm_copy = norm_copy.apply(z_scale, axis="columns")
    return norm_copy

def z_scale(row):
    sd = row.std()
    mean = row.mean()
    if sd > 0: 
        return pd.Series((row-mean)/sd)
    else:
        return pd.Series([0]*len(row))



path_to_csv = "/net/data.isilon/ag-cherrmann/nschmidt/project/parse_xml_for_VAE/xml_data/Aggregated_thalamic_nuclei.csv"

df = pd.read_csv(path_to_csv, header=[0, 1], index_col=0)

df_norm = normalize_and_scale_df(df=df)
df_norm


In [ ]:
data = np.zeros((5, 4))
columns = pd.MultiIndex.from_tuples([('A', 'val1'), ('B', 'val1'), ('C', 'val1'), ('D', 'val1')], names = ["Filename", "Volume"])
df_multi = pd.DataFrame(data, columns=columns)
df_multi.index = [["a", "b", "c", "d", "e"]]

# Slicing a DataFrame with MultiIndex Header

print(df_multi)
cols = df_multi.columns.get_level_values(-1).tolist()
unique_cols = list(set(cols))
cols_to_scale = [col for col in df_multi.columns if col[-1] == col_type] 


df_multi[[("A","val1"), ("B","val1")]] = df_multi[[("A","val1"), ("B","val1")]].apply(lambda x: pd.Series((x-x.mean())/x.std()) if x.std() > 0 else pd.Series([0.0]*len(x)), axis="columns")
df_multi


In [ ]:
#PCA - per atlas / split for volume kind (vgm/vwm/csf):

#general PCA function for one csv:
def PCA_plot_Aggregated_data(csv_path: str, n_components: int = 2, fillna_value: float = 0.0, n_clusters: int = 3):
    
    df = pd.read_csv(csv_path, header=0, index_col=[0, 1])  # !! -> transposed csv

    #amount volume categories -> plot size
    volume_kinds = df.index.get_level_values('Volume').unique()
    n = len(volume_kinds)
    cols = 3 
    rows = int(np.ceil(n / cols)) 
    # Erstelle Subplots basierend auf der Anzahl der Volume-Typen
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 6, rows * 5))
    axes = axes.flatten()

    #PCA per volume kind
    for idx, volume_kind in enumerate(volume_kinds):
    
        df_volume = df.loc[df.index.get_level_values('Volume') == volume_kind]
        df_volume.index = [f"{patient}/{tissue}" for patient, tissue in df_volume.index]
        # PCA
        pca = PCA(n_components=n_components)
        pca_result = pca.fit_transform(df_volume)
        # KMeans
        kmeans = KMeans(n_clusters=n_clusters)
        labels = kmeans.fit_predict(pca_result)
        # Plotting
        ax = axes[idx]
        scatter = ax.scatter(pca_result[:, 0], pca_result[:, 1], c=labels, cmap='viridis', alpha=0.6)
        ax.set_title(f"{os.path.basename(csv_path).replace('_t.csv', '').replace('Aggregated_','')} / {volume_kind}")
        ax.set_xlabel("PC1")
        ax.set_ylabel("PC2")
        cbar = plt.colorbar(scatter, ax=ax)
        cbar.set_label('Cluster')

    for i in range(n, len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()

    return fig, axes


In [ ]:
#PCA function application to all the atlases -> one fig for each
def apply_pca_to_all_csv_in_folder(folder_path: str, n_components: int = 2, fillna_value: float = 0.0, n_clusters: int = 3):
   
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    for idx, csv_file in enumerate(csv_files):
        csv_path = os.path.join(folder_path, csv_file)
        
        PCA_plot_Aggregated_data(csv_path, n_components, fillna_value, n_clusters)

    plt.tight_layout()
    plt.show()

In [ ]:
#simple correlation plots ROIS against ea

def correlation_heatmap_Aggregated_data(csv_path: str, fillna_value: float = 0.0):
    df = pd.read_csv(csv_path, header=0, index_col=[0, 1])  # !! -> transposed csv

    #separate Vwm/Wgm/csf
    volume_kinds = df.index.get_level_values('Volume').unique()
    n = len(volume_kinds)
    cols = 3
    rows = int(np.ceil(n / cols))  
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 6, rows * 5))
    axes = axes.flatten()

    #correlation per volume category
    for idx, volume_kind in enumerate(volume_kinds):
        df_volume = df.loc[df.index.get_level_values('Volume') == volume_kind]

        correlation_matrix = df_volume.corr()

        ax = axes[idx]
        sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', vmin=-1, vmax=1, center=0, square=True, ax=ax)
        ax.set_title(f"{os.path.basename(csv_path).replace('_t.csv', '').replace('Aggregated_','')} / {volume_kind}", fontsize=10)
        ax.set_xlabel("Features", fontsize=8)
        ax.set_ylabel("Features", fontsize=8)

        ax.tick_params(axis='both', which='major', labelsize=6)

    for i in range(n, len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()

    return fig, axes

# application to all atlases
def apply_correlation_heatmap(folder_path: str, fillna_value: float = 0.0):
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    for csv_file in csv_files:
        csv_path = os.path.join(folder_path, csv_file)
        correlation_heatmap_Aggregated_data(csv_path, fillna_value)


In [ ]:
apply_correlation_heatmap("/net/data.isilon/ag-cherrmann/nschmidt/project/parse_xml_for_VAE/xml_data_t")

In [ ]:

#numerical correlation comparison
def correlation_Aggregated_data(csv_path: str, fillna_value: float = 0.0):
    df = pd.read_csv(csv_path, header=0, index_col=[0, 1])  
    volume_kinds = df.index.get_level_values('Volume').unique()
    correlation_values_dict = {}

    for idx, volume_kind in enumerate(volume_kinds):
        df_volume = df.loc[df.index.get_level_values('Volume') == volume_kind]
        correlation_matrix = df_volume.corr() 
        correlation_matrix_name = f"{os.path.basename(csv_path).replace('_t.csv', '').replace('Aggregated_','')}_{volume_kind}_correlation_matrix"
        correlation_values_dict[correlation_matrix_name] = correlation_matrix 
    
    return correlation_values_dict

def get_top_correlations(correlation_matrix, top_n=5):
    upper_tri = correlation_matrix.where(
        np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
    )
    sorted_corrs = upper_tri.unstack().dropna().sort_values()
    smallest = sorted_corrs.head(top_n)
    largest = sorted_corrs.tail(top_n)
    return smallest, largest

def print_top_correlations(dict_all_atlases, top_n=5):
    for name, matrix in dict_all_atlases.items():
        print(f"\nTop {top_n} Correlations in {name}:")
        smallest, largest = get_top_correlations(matrix, top_n)

        print("\n Smallest Correlations:")
        print(smallest)

        print("\n Largest Correlations (≠1):")
        print(largest)


In [ ]:

folder_path = "/net/data.isilon/ag-cherrmann/nschmidt/project/parse_xml_for_VAE/xml_data_t"

dict_all_atlases = {}
for file in os.listdir(folder_path):
    if file.endswith(".csv"):
        full_path = os.path.join(folder_path, file)
        result = correlation_Aggregated_data(full_path)
        dict_all_atlases.update(result)
        
print_top_correlations(dict_all_atlases, top_n=5)


PROBLEM: 
-> largest correlations are often due to respective areas in left & right hemisphere 
-> more interesting are areas that are not bilateral equivalent 

In [ ]:
#correlation again !ignoring symmetrical pairs!
import re
def clean_roi_name(roi):
    #removes l or r in name -> neutral name
    return re.sub(r'^[lr]', '', roi)

def get_top_correlations_filtered(correlation_matrix, top_n=5):
    upper_tri = correlation_matrix.where(
        np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
    )
    sorted_corrs = upper_tri.unstack().dropna().sort_values()

    #only non-equal pairs
    filtered_corrs = sorted_corrs[
        [(clean_roi_name(i) != clean_roi_name(j)) for i, j in sorted_corrs.index]
    ]

    smallest = filtered_corrs.head(top_n)
    largest = filtered_corrs.tail(top_n)

    return smallest, largest

def print_top_correlations_nosym(dict_all_atlases, top_n=5):
    for name, matrix in dict_all_atlases.items():
        print(f"\n📊 Top {top_n} Correlations in {name} (non bilateral):")
        smallest, largest = get_top_correlations_filtered(matrix, top_n)

        print("\n🔻 Smallest Correlations:")
        print(smallest)

        print("\n🔺 Largest Correlations:")
        print(largest)


In [ ]:
folder_path = "/net/data.isilon/ag-cherrmann/nschmidt/project/parse_xml_for_VAE/xml_data_t"

dict_all_atlases = {}
for file in os.listdir(folder_path):
    if file.endswith(".csv"):
        full_path = os.path.join(folder_path, file)
        result = correlation_Aggregated_data(full_path)
        dict_all_atlases.update(result)
        
print_top_correlations_nosym(dict_all_atlases, top_n=5)